In [1]:
import numpy as np
from modules import bayesianL
from modules import cubicharmonics
import matplotlib.pyplot as plt
import copy

In [39]:
#Carico i dati in data e li ordino in sdata e grid

data=np.transpose(np.loadtxt('../nlttk.out'))

sdata, grid = bayesianL.generatesorteddata(data, 500)

[1. 0. 0.] [0. 1. 0.] [0. 0. 1.]


In [40]:
#Medio i dati su tutti i punti equivalenti per simmetria cubica e 
#ritorno la lista dei nuovi dati 'simmetrizzati' in gplot e sd
C=bayesianL.cubicarray(list(grid), True)
dic, dics=bayesianL.datadicG(data[1], data[2],cubicharmonics.Gvecgenerateall(500)[1:])
gplot, dataplot, datasigmaplot=bayesianL.datiplot(C, dic, dics)
sd=np.array([dataplot, datasigmaplot*np.sqrt(10.0)])

numero di elementi:  499
numero cicli teornicamente necessari 373501
numero cicli compiuti 381
numero di elementi non equivalenti per simmetria cubica:  118


In [41]:
#Calcolo il fit bayesiano con glot e sd come test
N=50
mN, SN, y_infer, sy_infer, conta=\
bayesianL.bayesianpol(gplot, sd, 16, N, 0.005, gplot, ifprint= True, ifwarning= True, nLbp=0)

parametri ottimali [  0.57055612  -9.80528729  28.7674318  -29.64846911  -6.21045818
  11.67212788  11.68188406   4.31951931  -2.42731439  -5.82661815
  -6.08924568  -4.3805157   -1.83946851   0.73949238   2.91765241
   4.52099906]
numero di armoniche cubiche 30
valore a kmin inferito 0.5228938011693693 bias 0.5705561239624863 dato a kmin 0.7147155658198961
determinante matrice delle armoniche cubiche ridotte: -1.9231635169287893e-87


In [42]:
#Plotto il fit e i dati (gplot e sd)
C=bayesianL.cubicarray(list(grid), True)
dic, dics=bayesianL.datadicG(data[1], data[2],cubicharmonics.Gvecgenerateall(500)[1:])
gplot, dataplot, datasigmaplot=bayesianL.datiplot(C, dic, dics)
%matplotlib widget

plt.fill_between(np.linalg.norm(gplot[:N], axis=1), y_infer+np.sqrt(sy_infer), y_infer-np.sqrt(sy_infer),  alpha=0.3, color='red')
plt.plot(np.linalg.norm(gplot[:N], axis=1), y_infer, '.-', color='red')
plt.errorbar(np.linalg.norm(gplot, axis=1), dataplot,datasigmaplot*np.sqrt(10.), fmt='.')

numero di elementi:  499
numero cicli teornicamente necessari 373501
numero cicli compiuti 381
numero di elementi non equivalenti per simmetria cubica:  118


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ErrorbarContainer object of 3 artists>

In [43]:
#Calcolo il best fit su gplot e sd con 50 dati
N=50
mN, SN, y_infer, sy_infer, spar, log_evidence_vP=\
bayesianL.bestfit(gplot, sd, N, gplot, ifprintbestfit= False, ifprintfinal= True, nLbf=4)
#Plotto il best fit e i dati (gplot e sd)
C=bayesianL.cubicarray(list(grid), True)
dic, dics=bayesianL.datadicG(data[1], data[2],cubicharmonics.Gvecgenerateall(500)[1:])
gplot, dataplot, datasigmaplot=bayesianL.datiplot(C, dic, dics)
%matplotlib widget

plt.fill_between(np.linalg.norm(gplot[:N], axis=1), y_infer+np.sqrt(sy_infer), y_infer-np.sqrt(sy_infer),  alpha=0.3, color='red')
plt.plot(np.linalg.norm(gplot[:N], axis=1), y_infer, '.-', color='red')
plt.errorbar(np.linalg.norm(gplot, axis=1), dataplot,datasigmaplot*np.sqrt(10.), fmt='.')

parametri ottimali [ 0.50291966 -4.55400344  0.18573122 -0.29509677  0.18308196 -0.21607441
  5.65641415 -0.98115281  1.50636261 -1.30742512  1.26423444 -0.49544001
  1.63144454 -2.30461759  2.74760712 -2.33068399 -2.1760961  -0.87713116
  1.08208982 -1.80138455  1.36948841]
numero di armoniche cubiche 5
valore a kmin inferito 0.4729027008714883 bias 0.5029196611986521 dato a kmin 0.7147155658198961
determinante matrice delle armoniche cubiche ridotte: 5.408130296567605e+44
grado ottimale 10 grado massimo tentato 28
numero di polinomi nella base ottimale:  21 numero di dati 50
best alpha 0.19003311677969964
numero di elementi:  499
numero cicli teornicamente necessari 373501
numero cicli compiuti 381
numero di elementi non equivalenti per simmetria cubica:  118


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ErrorbarContainer object of 3 artists>

In [44]:
#Plotto la funzione di evidence che ho massimizzato per il best fit nella cella precedente
%matplotlib widget
plt.plot(range(2,(len(log_evidence_vP)+1)*2, 2), log_evidence_vP, '.')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [45]:
#Calcolo il best fit includendo sempre piu' punti 
predbl=[]
spredbl=[]
Npointsbl=[]
for i in range(5):
    predb=[]
    spredb=[]
    Npointsb=[]
    for N in range(5, 56):
        mN, SN, y_infer, sy_infer, spar, log_evidence_vP=\
        bayesianL.bestfit(gplot, sd, N, gplot, ifprintbestfit=False, ifprintfinal=False, nLbf=i)
        Npointsb.append(np.linalg.norm(gplot[N]))
        if (N-5)%(50//5)==0: print(int((N-5)/50*100), '%')
        predb.append(mN[0])
        spredb.append(SN[0,0])
    predbl.append(predb)
    spredbl.append(spredb)
    Npointsbl.append(Npointsb)

0 %
20 %
40 %
60 %
80 %
100 %
0 %
20 %
40 %
60 %
80 %
100 %
0 %
20 %
40 %
60 %
80 %
100 %
0 %
20 %
40 %
60 %
80 %
100 %
0 %
20 %
40 %
60 %
80 %
100 %


In [50]:
%matplotlib widget
fig, ax=plt.subplots(2,constrained_layout=True, figsize=[9,9])


#ax[0].set_title('Bayesian model averaging prediction at k=0 with data in range $[: k_{max}]$')
#
#ax[0].errorbar(np.array(Npointsa), np.array(preda), np.sqrt(np.array(spreda)), fmt='.')
#ax[0].plot(np.array(Npointsa), np.ones(len(Npointsa))*0.816, label='GK result')
#    
#ax[0].set_xlabel('$k_{max} (nm^{-1})$')
#ax[0].set_ylabel('prediction at k=0')
#ax[0].legend()
ax[0].set_title('Bayesian best model prediction at k=0 with data in range $[: k_{max}]$')
L=[0,[0,4],[0,4,6], [0,4,6,8], [0,4,6,8,10]]
for i in range(5):
    ax[0].errorbar(np.array(Npointsbl[i]), np.array(predbl[i]), np.sqrt(np.array(spredbl[i])), fmt='.', label=r'$L_{max}$'+'={}'.format(L[i]))
ax[0].plot(np.array(Npointsbl[1]), np.ones(len(Npointsbl[1]))*0.816, label='GK result')    
ax[0].set_xlabel('$k_{max} (nm^{-1})$')
ax[0].set_ylabel('prediction at k=0')
ax[0].legend()
ax[1].set_title('Data')

ax[1].errorbar(np.linalg.norm(gplot, axis=1), dataplot,datasigmaplot*np.sqrt(10.), fmt='.', label='3375 water molecules at 300k 30ns') 
ax[1].plot(np.linalg.norm(gplot, axis=1), np.ones(len(gplot))*0.816, label='GK result') 
ax[1].legend()

ax[1].set_xlabel(r'k ($nm^{-1}$)')
plt.savefig('testLnltt.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [28]:
#Calcolo il best fit includendo sempre piu' punti 
predbl=[]
spredbl=[]
Npointsbl=[]
for i in range(5):
    predb=[]
    spredb=[]
    Npointsb=[]
    for N in range(5, 56):
        mN, SN,log_evidence_vP=\
        bayesianL.bayesianmodelprediction(gplot, sd, N, gplot, ifprintmodpred=False, ifprintfinal=False, nLbmp=i)
        Npointsb.append(np.linalg.norm(gplot[N]))
        if (N-5)%(50//5)==0: print(int((N-5)/50*100), '%')
        predb.append(mN)
        spredb.append(SN)
    predbl.append(predb)
    spredbl.append(spredb)
    Npointsbl.append(Npointsb)

0 %
20 %
40 %
60 %
80 %
100 %
0 %
20 %
40 %
60 %
80 %
100 %
0 %
20 %
40 %
60 %
80 %
100 %
0 %
20 %
40 %
60 %
80 %
100 %
0 %
20 %
40 %
60 %
80 %
100 %


In [29]:
%matplotlib widget
fig, ax=plt.subplots(2,constrained_layout=True, figsize=[9,9])


#ax[0].set_title('Bayesian model averaging prediction at k=0 with data in range $[: k_{max}]$')
#
#ax[0].errorbar(np.array(Npointsa), np.array(preda), np.sqrt(np.array(spreda)), fmt='.')
#ax[0].plot(np.array(Npointsa), np.ones(len(Npointsa))*0.816, label='GK result')
#    
#ax[0].set_xlabel('$k_{max} (nm^{-1})$')
#ax[0].set_ylabel('prediction at k=0')
#ax[0].legend()
ax[0].set_title('Bayesian best model prediction at k=0 with data in range $[: k_{max}]$')
L=[0,[0,4],[0,4,6], [0,4,6,8], [0,4,6,8,10]]
for i in range(5):
    ax[0].errorbar(np.array(Npointsbl[i]), np.array(predbl[i]), np.sqrt(np.array(spredbl[i])), fmt='.', label=r'$L_{max}$'+'={}'.format(L[i]))
ax[0].plot(np.array(Npointsbl[1]), np.ones(len(Npointsbl[1]))*0.816, label='GK result')    
ax[0].set_xlabel('$k_{max} (nm^{-1})$')
ax[0].set_ylabel('prediction at k=0')
ax[0].legend()
ax[1].set_title('Data')

ax[1].errorbar(np.linalg.norm(gplot, axis=1), dataplot,datasigmaplot*np.sqrt(10.), fmt='.', label='3375 water molecules at 300k 30ns') 

ax[1].legend()

ax[1].set_xlabel(r'k ($nm^{-1}$)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'k ($nm^{-1}$)')